In [41]:
import pandas as pd
import math
import numpy as np
import sklearn.cluster

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

data = pd.read_csv(r"../data/full_data.csv", dtype={"test_units":str})
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')

c:\users\xoxix\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (33,61,62,93) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [45]:
triggers = {"C1": 3, "C2": 3, "C4": 4}

severity_triggers = pd.DataFrame({"iso_code": data["iso_code"].unique(), "continent": "None"})

for key in triggers:
    severity_triggers[key] = 6.0

for index, row in severity_triggers.iterrows():
    continent_data = data["continent"][data["iso_code"] == row["iso_code"]]
    if continent_data.shape[0] > 0:
        severity_triggers.at[index, "continent"] = continent_data.iloc[0]
    for key in triggers:
        measure_dates = data["date"][(data["iso_code"] == row["iso_code"]) & (data[key + "_combined_numeric"] == triggers[key])]
        if measure_dates.shape[0] > 0:
            new_value = data["reproduction_rate"][(data["iso_code"] == row["iso_code"]) & (data["date"] == min(measure_dates))].mean()
            if (not math.isnan(new_value)):
                severity_triggers.at[index, key] = new_value

In [51]:
k_means = sklearn.cluster.KMeans(n_clusters = 3)
severity_triggers["policy_cluster"] = k_means.fit_predict(severity_triggers[triggers.keys()])
pd.crosstab(severity_triggers["policy_cluster"], severity_triggers["continent"])

continent,Africa,Asia,Europe,North America,Oceania,South America
policy_cluster,,,,,,
0,32,18,9,14,5,8
1,5,20,26,6,1,2
2,14,9,8,1,2,2


In [52]:
k_means.cluster_centers_

array([[5.89988372, 5.92430233, 6.        ],
       [4.4265    , 1.60583333, 2.76083333],
       [5.46361111, 6.        , 1.29527778]])

In [10]:
import numpy as np

np.nanmax(data["reproduction_rate"])

5.79